In [3]:
# Widgets
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual

# We'll use pandas to generate a nice table
import pandas as pd

# We'll graph the functions, so we'll need the matplotlib library.
# We'll also choose the display mode for these graphs. To simplify, we choose
%matplotlib inline
# Now, we'll nickname the pyplot submodule of matplotlib as plt.
import matplotlib.pyplot as plt

# Finally, we'll import some LaTeX and Markdown display helpers
from IPython.display import display, clear_output, Markdown, Latex

# How many minutes?

In TechTown, there are two cell phone service providers. "PhoneWorld" offers a monthly plan, with a \\$35 fee for 120 minutes, plus \\$0.10 each additional minute. "CellUniverse" offers a \\$30 monthly plan for 100 minutes, and each additional minute costs \\$0.15 up to 300 minutes. After that, each extra minute costs \\$0.05. 

How many monthly minutes would you have to use such that "PhoneWorld" is the best option, that is, costs less than "CellUniverse"?

## Setting up the problem (inspection)

First, let's take a look at how we could compute the monthly fee for each cell phone service provider. Usually when looking at problems like this, we try some random values at it. Let's try a table:

In [4]:
def phoneworld(minutes):
    if minutes <= 120:
        cost = 35
    else:
        cost = 35+(minutes-120)*0.1
    return cost

In [5]:
def celluniverse(minutes):
    if minutes <= 100:
        cost = 30
    elif minutes <= 300:
        cost = 30 + (minutes-100)*0.15
    else:
        cost = 30 + 200*0.15 + (minutes-300)*0.05
    return cost

In [13]:
minutes_input = widgets.Text(
                    value='',
                    placeholder='Enter minutes (decimal)',
                    description="Minutes =",
                )

# Initializing table with the first row
dados = [['Basic Fee', 35, 30]] 
table = pd.DataFrame(dados, columns = ['Minutes', 'PhoneWorld', 'CellUniverse']) 

interact_manual.opts['manual_name'] = 'Compute Cost'

@interact_manual(minutes=minutes_input, table=fixed(table))
def update_table(minutes, table):
    if minutes != '':
        time = float(minutes)
        if len(table[table['Minutes'] == time]) > 0:
            # this value has been computed already, don't do anything
            pass
        else:
            table.loc[len(table)+1] = (time, phoneworld(time), celluniverse(time))
    # we'll clear the table index to have a nicer display
    table.index = ['']*len(table)
    display(table)

interactive(children=(Text(value='', description='Minutes =', placeholder='Enter minutes (decimal)'), Button(d…

Now, it would be interesting to see how these values compare in a graph. In the horizontal axis, we'll mark how many minutes were used, and in the vertical axis we'll mark the corresponding cost for each company.

In [14]:
fig, ax = plt.subplots()
plt.close(fig)

graph_button = widgets.Button(
                description='Graph computed values',
                tooltip='Graph computed values',
                layout=widgets.Layout(width='auto')
                )

connect_button = widgets.Button(
                    description='Connect points',
                    tooltip='Connect points',
                    layout=widgets.Layout(width='auto')
                    )

display(widgets.HBox([graph_button, connect_button]))
out_two_graphs = widgets.Output()
display(widgets.HBox(children=(out_two_graphs,)))

def graph_computed(graph):
    ax.clear()
    ax.plot(table['Minutes'][1:], table['PhoneWorld'][1:], 'r*', label="PhoneWorld")
    ax.plot(table['Minutes'][1:], table['CellUniverse'][1:], 'bo', label="CellUniverse")
    ax.legend()
    with out_two_graphs:
        clear_output(wait=True)
        display(fig)
        
def connect_points(connect):
    ax.clear()
    in_values = table[1:].sort_values(by='Minutes')
    ax.plot(in_values['Minutes'], in_values['PhoneWorld'], 'r*')
    ax.plot(in_values['Minutes'], in_values['CellUniverse'], 'bo')
    ax.plot(in_values['Minutes'], in_values['PhoneWorld'], 'r')
    ax.plot(in_values['Minutes'], in_values['CellUniverse'], 'b')
    ax.legend(("PhoneWorld", "CellUniverse"))
    with out_two_graphs:
        clear_output(wait=True)
        display(fig)

graph_button.on_click(graph_computed)
connect_button.on_click(connect_points)

Now, we can see that there is an interval where the red line is lower than the blue line; that means that PhoneWorld is less expensive than CellUniverse only in that interval. Let's see when that is.

From the graph, we can see that the interval that we are interested in is determined by the two points where the blue and the red line intersect. Now, how do we find those points?

## Finding the intersection of the two lines

Both lines are determined by *functions*. These are a way to connect the minutes to their correponding costs for each company. Just to keep it simpler, we'll give a nickname to the monthly cost of each company. The cost for PhoneWorld will be called $W$ and the cost for CellUniverse will be called $U$. 

We can see that both costs are determined by how many minutes we used in each month. So we can say something like this:

In [15]:
table.set_index('Minutes')
for index, row in table[1:].iterrows():
    print('For {0} minutes, we have W({0}) = {1}, U({0}) = {2}.'.format(int(row[0]), row[1], row[2]))

For 100 minutes, we have W(100) = 35.0, U(100) = 30.0.
For 120 minutes, we have W(120) = 35.0, U(120) = 33.0.
For 150 minutes, we have W(150) = 38.0, U(150) = 37.5.
For 200 minutes, we have W(200) = 43.0, U(200) = 45.0.
For 250 minutes, we have W(250) = 48.0, U(250) = 52.5.
For 300 minutes, we have W(300) = 53.0, U(300) = 60.0.
For 500 minutes, we have W(500) = 73.0, U(500) = 70.0.
For 450 minutes, we have W(450) = 68.0, U(450) = 67.5.


Notice that for values that are not on the table, we can't directly decide how much it would cost, unless we compute this number too. Fortunately, there is an easier way to describe to total cost without us having to list all possible values: using the *law* of the function. In our case, we have the following: let us call $t$ the total amount of minutes used in that month. Then,

In [16]:
# Title
display(Markdown('For PhoneWorld: (Press Enter after typing)'))

# Display function name - W(t)
name_pw = widgets.Output(layout=widgets.Layout(width='80px'))
with name_pw:
    display(Latex('$W(t)= $'))
    
# Display first interval for the function definition
cond1_pw = widgets.Output(layout=widgets.Layout(width='auto', grid_area='cond1_pw'))
with cond1_pw:
    display(Latex('for $t \leq 120$'))
    
# Display second interval for the function definition
cond2_pw = widgets.Output(layout=widgets.Layout(width='auto', grid_area='cond2_pw'))
with cond2_pw:
    display(Latex('for $t > 120$.'))
    
# Display resulting function graph
fig_pw, ax_pw = plt.subplots()
outgraph_pw = widgets.Output()
plt.close(fig_pw)

def start_fig_pw(fig_pw, ax_pw, outgraph_pw):
    with outgraph_pw:
        values = [0, 120, 500]
        pw = []
        for t in values:
            pw.append(phoneworld(t))
        ax_pw.plot(values, pw, 'r', label='PhoneWorld')
        ax_pw.legend()
        display(fig_pw)

start_fig_pw(fig_pw, ax_pw, outgraph_pw)

# Widgets for the function law input
box1_pw = widgets.Text(layout=widgets.Layout(width='150px', grid_area='box1_pw'))
box2_pw = widgets.Text(layout=widgets.Layout(width='150px', grid_area='box2_pw'))

# Widgets are displayed in a grid
grid_pw = widgets.GridBox(children=[box1_pw, cond1_pw, box2_pw, cond2_pw],
            layout=widgets.Layout(
            width='350px',
            grid_template_rows='auto auto',
            grid_template_columns='150px 200px',
            grid_template_areas='''
            "box1_pw cond1_pw"
            "box2_pw cond2_pw"
            ''')
       )

# This widget contains the three previously defined widgets, side by side.
horiz_box = widgets.HBox(children=[name_pw, grid_pw, outgraph_pw],
                    layout=widgets.Layout(width='90%', align_items='center', justify_content='center'))

# The functions below show the graph of the conditions given by the users 
# in the input boxes defined above. The graphs are then compared to the 
# true function graph, so that we can check that the suggested law actually
# represents the conditions we want to observe for this function.
def graph_pw_cond1(box):
    values = [0, 120]
    cost = []
    for t in values:
        # careful with eval!
        cost.append(eval(box.value))
    ax_pw.plot(values, cost, 'k-', label="My graph ($0\leq t\leq 120$)")
    ax_pw.legend()
    with outgraph_pw:
        clear_output(wait=True)
        display(fig_pw)
        
def graph_pw_cond2(box):
    values = [120, 500]
    cost = []
    for t in values:
        cost.append(eval(box.value))
    ax_pw.plot(values, cost, 'k-', label="My graph ($t>120$)")
    ax_pw.legend()
    with outgraph_pw:
        clear_output(wait=True)
        display(fig_pw)
    
#####
# Finally, we display all the widgets
display(horiz_box)
box1_pw.on_submit(graph_pw_cond1)
box2_pw.on_submit(graph_pw_cond2)

# We create a button to clear the graph in case we make a mistake
clear_button_pw = widgets.Button(
                description='Clear graph',
                tooltip='Clear graph',
                layout=widgets.Layout(width='auto')
                )

def clear_graph_pw(button):
    with outgraph_pw:
        clear_output(wait=True)
        ax_pw.clear()
        start_fig_pw(fig_pw, ax_pw, outgraph_pw)

display(clear_button_pw)
clear_button_pw.on_click(clear_graph_pw)

For PhoneWorld: (Press Enter after typing)

Button(description='Clear graph', layout=Layout(width='auto'), style=ButtonStyle(), tooltip='Clear graph')

Let's repeat this for CellUniverse (remember that here we have 3 price ranges):

In [7]:
# Title
display(Markdown('For CellUniverse: (Press Enter after typing)'))

# Display function name - U(t)
name_cu = widgets.Output(layout=widgets.Layout(width='80px'))
with name_cu:
    display(Latex('$U(t)= $'))
    
# Display first interval for the function definition
cond1_cu = widgets.Output(layout=widgets.Layout(width='auto', grid_area='cond1_cu'))
with cond1_cu:
    display(Latex('for $t \leq 100$'))
    
# Display second interval for the function definition
cond2_cu = widgets.Output(layout=widgets.Layout(width='auto', grid_area='cond2_cu'))
with cond2_cu:
    display(Latex('for $100 < t \leq 300$'))

# Display third interval for the function definition
cond3_cu = widgets.Output(layout=widgets.Layout(width='auto', grid_area='cond3_cu'))
with cond3_cu:
    display(Latex('for $t > 300$.'))

# Display resulting function graph
fig_cu, ax_cu = plt.subplots()
plt.close(fig_cu)
outgraph_cu = widgets.Output()

def start_fig_cu(fig_cu, ax_cu, outgraph_cu):
    with outgraph_cu:
        values = np.linspace(0,500,50)
        cu = []
        for t in values:
            cu.append(celluniverse(t))
        ax_cu.plot(values, cu, 'b', label='CellUniverse')
        ax_cu.legend()
        display(fig_cu)

start_fig_cu(fig_cu, ax_cu, outgraph_cu)
    
# Widgets for the function law input
box1_cu = widgets.Text(layout=widgets.Layout(width='150px', grid_area='box1_cu'))
box2_cu = widgets.Text(layout=widgets.Layout(width='150px', grid_area='box2_cu'))
box3_cu = widgets.Text(layout=widgets.Layout(width='150px', grid_area='box3_cu'))

# Widgets are displayed in a grid
grid_cu = widgets.GridBox(children=[box1_cu, cond1_cu, box2_cu, cond2_cu, box3_cu, cond3_cu],
            layout=widgets.Layout(
            width='550px',
            grid_template_rows='auto auto auto',
            grid_template_columns='150px 200px 200px',
            grid_template_areas='''
            "box1_cu cond1_cu"
            "box2_cu cond2_cu"
            "box3_cu cond3_cu"
            ''')
       )

# This widget contains the three previously defined widgets, side by side.
horiz_cu = widgets.HBox(children=[name_cu, grid_cu, outgraph_cu],
                    layout=widgets.Layout(width='100%', align_items='center', justify_content='center'))

def graph_cu_cond1(box):
    values = np.linspace(0,100,50)
    boxin = []
    for t in values:
        boxin.append(eval(box.value))
    ax_cu.plot(values, boxin, 'k-', label="My graph ($0\leq t\leq 100$)")
    ax_cu.legend()
    with outgraph_cu:
        clear_output(wait=True)
        display(fig_cu)
        
def graph_cu_cond2(box):
    values = np.linspace(100,300,50)
    boxin = []
    for t in values:
        boxin.append(eval(box.value))
    ax_cu.plot(values, boxin, 'k-', label="My graph ($100<t\leq 300$)")
    ax_cu.legend()
    with outgraph_cu:
        clear_output(wait=True)
        display(fig_cu)

def graph_cu_cond3(box):
    values = np.linspace(300,500,50)
    boxin = []
    for t in values:
        boxin.append(eval(box.value))
    ax_cu.plot(values, boxin, 'k-', label="My graph ($t > 500$)")
    ax_cu.legend()
    with outgraph_cu:
        clear_output(wait=True)
        display(fig_cu)

display(horiz_cu)
box1_cu.on_submit(graph_cu_cond1)
box2_cu.on_submit(graph_cu_cond2)
box3_cu.on_submit(graph_cu_cond3)

clear_button_cu = widgets.Button(
                description='Clear graph',
                disabled=False,
                button_style='',
                tooltip='Clear graph',
                layout=widgets.Layout(width='200px')
                )

def clear_graph_cu(clear):
    with outgraph_cu:
        clear_output(wait=True)
        ax_cu.clear()
        start_fig_cu(fig_cu, ax_cu, outgraph_cu)

display(clear_button_cu)
clear_button_cu.on_click(clear_graph_cu)

For CellUniverse:

Button(description='Clear graph', layout=Layout(width='200px'), style=ButtonStyle(), tooltip='Clear graph')

Now, you may have seen that the laws for W(t) and U(t) are:

In [8]:
out_w = widgets.Output()
out_v = widgets.Output()
with out_w:
    display(Latex(r'$$W(t) = \begin{cases} 35, & t \leq 120\\ 35+0.1(t-120), & t> 120\end{cases}$$'))
with out_v:
    display(Latex(r'$$U(t) = \begin{cases} 30, & t\leq 100\\30+0.15(t-100), &100<t\leq 300\\ 30 + 200(0.15) + 0.05(t-300), & t>300.\end{cases}$$'))
accordion = widgets.Accordion(children=[out_w, out_v], selected_index = None)
accordion.set_title(0, 'PhoneWorld:')
accordion.set_title(1, 'CellUniverse:')
display(accordion)

Accordion(children=(Output(), Output()), selected_index=None, _titles={'0': 'PhoneWorld:', '1': 'CellUniverse:…

Finally, to find the intersection points between the two curves, we can see that for the leftmost point, we are in the interval where $100<t<300$, and so we will use the second law for $W$ and the second law for $U$:

\begin{align*}
    35+0.1(t-120) &= 30+0.15(t-100)\\
    35 + 0.1t - 12 &= 30+0.15t - 15\\
    0.1t-0.15t &= 30-15-35+12\\
    -0.05t &= -8
\end{align*}
and so,
$$t = \frac{800}{5} = 160.$$

In fact, we can check that this is true: $W(160)$ is 

In [9]:
phoneworld(160)

39.0

and $U(160)$ is 

In [10]:
celluniverse(160)

39.0

Next, we have a point where $400<t<500$, and so we must have

\begin{align*}
    35+0.1(t-120) &= 30 + 200(0.15) + 0.05(t-300)\\
    35 + 0.1t - 12 &= 30 + 30 + 0.05t - 15\\
    0.1t-0.05t &= 30+30-15-35+12\\
    0.05t &= 22
\end{align*}
and so,
$$t = \frac{2200}{5} = 440.$$

Indeed, $W(440)$ is 

In [11]:
phoneworld(440)

67.0

and $U(440)$ is 

In [12]:
celluniverse(440)

67.0

## Answer

For PhoneWorld to be the best option, our monthly usage must be between 160 and 440 minutes.